### Russel 2000
---

##### Explication de la Méthode Adoptée

En tentant de récupérer les données des composants de l'indice Russell 2000, nous avons rencontré des difficultés à trouver une source directement scrapable. Les sites web souvent utilisés pour ces informations, comme les sites financiers spécialisés, présentent des défis comme le chargement dynamique des données via JavaScript, qui rend le scraping direct complexe ou même impossible sans outils avancés comme Selenium.

Face à ces obstacles, nous avons opté pour une méthode alternative :
1. **Téléchargement initial des données** : Nous avons d'abord téléchargé une base de données complète des stocks du Russell 2000 depuis une source fiable et accessible, le site [Sure Dividend](https://www.suredividend.com/russell-2000-stocks/), qui propose des listes régulièrement mises à jour.
2. **Sauvegarde sur Google Drive** : Pour assurer la reproductibilité et l'accès facile à ces données sans les contraintes du scraping direct, nous avons sauvegardé cette base de données sur Google Drive. Un lien de partage public a été créé pour permettre un accès universel.
3. **Utilisation de Google Sheets** : Plutôt que de scraper un site web, nous avons chargé les données dans Google Sheets et utilisé l'exportation CSV de Google Sheets pour faciliter l'accès programmatique aux données via des requêtes HTTP simples, exploitées ensuite avec Python.

Cette approche garantit non seulement une plus grande stabilité et reproductibilité dans l'accès aux données mais contourne également les éventuels problèmes légaux ou techniques liés au scraping direct de sites web.


In [1]:
import pandas as pd
import requests
from io import StringIO
import yfinance as yf

In [2]:
def load_google_sheet_csv(url):
    """
    Charge un Google Sheet exporté en CSV depuis une URL et le retourne sous forme de DataFrame.

    Paramètres:
    - url (str): URL complète du fichier Google Sheet exporté en CSV.

    Retour:
    - DataFrame contenant les données du fichier CSV.

    Exceptions:
    - AssertionError pour les réponses HTTP non réussies.
    - Exception pour les erreurs de lecture CSV.
    """
    response = requests.get(url)
    assert response.status_code == 200, 'Erreur de téléchargement'
    try:
        # StringIO simule un fichier à partir d'une chaîne, nécessaire pour utiliser pd.read_csv
        return pd.read_csv(StringIO(response.text))
    except Exception as e:
        raise Exception(f"Erreur lors de la lecture CSV: {e}")

In [3]:
csv_url = "https://docs.google.com/spreadsheets/d/1OGMIF5aozD_O5r0WAhmANDizKXcsbd8x_OTjTPf-IcQ/export?format=csv&gid=0"
df_russel2000 = load_google_sheet_csv(csv_url)
df_russel2000.head()

,Ticker,Name,Sector,Price,Dividend Yield,5-Year Average Dividend Yield,Dividends Per Share (TTM),Market Cap ($M),Trailing P/E Ratio,Payout Ratio,Beta,52-Week High,52-Week Low
0,AADI,"Aadi Bioscience, Inc.",NaN,$2.46,NaN,0.0%,0.0%,$58.17,NaN,0.0,54.4%,5.5,$1.21
1,AAN,"Aaron's Company, Inc.",Industrials,$10.09,5.0%,3.4%,50.0%,$309.89,NaN,NaN,122.7%,NaN,NaN
2,AAON,"AAON, INC.",Industrials,$135.83,NaN,0.2%,32.0%,"$11,034.52",$57.80,0.0,116.9%,144.0,$62.65
3,AAT,"American Assets Trust, Inc.",Real Estate,$27.87,4.8%,4.3%,133.0%,"$1,694.89",$29.07,1.0,89.1%,29.0,$19.25
4,ABCB,Ameris Bancorp,Financial Services,$68.83,1.0%,0.9%,60.0%,"$4,756.63",$14.40,0.0,89.4%,74.6,$43.62


In [4]:
# On enlève l'information superflue
df_russel2000 = df_russel2000.drop(columns=[
    'Sector', 'Price', 'Dividend Yield', '5-Year Average Dividend Yield', 'Dividends Per Share (TTM)',
    'Trailing P/E Ratio', 'Payout Ratio', 'Beta', '52-Week High', '52-Week Low'
])
df_russel2000.head()

,Ticker,Name,Market Cap ($M)
0,AADI,"Aadi Bioscience, Inc.",$58.17
1,AAN,"Aaron's Company, Inc.",$309.89
2,AAON,"AAON, INC.","$11,034.52"
3,AAT,"American Assets Trust, Inc.","$1,694.89"
4,ABCB,Ameris Bancorp,"$4,756.63"


In [5]:
df_russel2000.rename(columns={'Name': 'Company Name'}, inplace=True)
df_russel2000.rename(columns={'Market Cap ($M)': 'Market Cap'}, inplace=True)

# Check des valeurs manquantes
df_russel2000.isnull().sum()

Ticker           0
Company Name     9
Market Cap      28
dtype: int64

In [6]:
# On supprime les lignes où 'Company name' est NaN 
# (après vérification, les données sont introuvables même sur yfinance)
df_russel2000 = df_russel2000.dropna(subset=['Company Name'])

In [7]:
def update_dataframe(df):
    """
    Met à jour la colonne 'Market Cap' du DataFrame :
    - Si 'Market Cap' est NaN, remplace par les données de yfinance.
    - Sinon, supprime le symbole '$' de 'Market Cap', le convertit en float * million, puis en entier.
    """
    for index, row in df.iterrows():
        # Mise à jour de 'Market Cap' si NaN
        if pd.isna(row['Market Cap']):
            ticker = row['Ticker']
            yf_ticker = yf.Ticker(ticker)
            market_cap = yf_ticker.info.get('marketCap')
            if market_cap:
                # Enregistrer la valeur entière
                df.at[index, 'Market Cap'] = int(market_cap)
            else:
                print(f"Erreur lors de la récupération du Market Cap pour {ticker}.")
       
        else:
            # Supprimer le symbole '$', convertir en float puis en entier
            market_cap_value = float(str(row['Market Cap']).replace('$', '').replace(',', '')) * 1e6
            df.at[index, 'Market Cap'] = int(market_cap_value)
    
    return df

In [8]:
# Application
df_russel2000 = update_dataframe(df_russel2000)

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/BIG?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=BIG&crumb=9dm%2FXU5uWNc


Erreur lors de la récupération du Market Cap pour BIG.


In [9]:
# On supprime les lignes où 'Market Cap' est encore NaN 
# (après vérification, les données sont introuvables même sur yfinance)
df_russel2000 = df_russel2000.dropna(subset=['Market Cap'])

# Check des valeurs manquantes
df_russel2000.isnull().sum()

Ticker          0
Company Name    0
Market Cap      0
dtype: int64

In [10]:
df_russel2000 = df_russel2000.sort_values(by = 'Market Cap', ascending = False)
df_russel2000

,Ticker,Company Name,Market Cap
1151,MSTR,MICROSTRATEGY Inc,78301730000
484,CVNA,CARVANA CO.,32863940000
1611,SMCI,"Super Micro Computer, Inc.",24090160000
677,FIX,COMFORT SYSTEMS USA INC,17694480000
551,DUOL,"Duolingo, Inc.",16407560000
...,...,...,...
1416,PRST,Presto Automation Inc.,1255557
778,GRPH,"Graphite Bio, Inc.",20000
1823,VAXX,"Vaxxinity, Inc.",12678
287,BRBS,Blue Ridge Bankshares Inc,0


In [11]:
# Exportation du DataFrame en CSV
file_path = "/Users/pieropelosi/Downloads/russell_2000.csv"
df_russel2000.to_csv(file_path, index=False)